# MACHINE LEARING PROJECT - SPEECH TO TEXT

# Ayesha Shariff - 21BDA18

In [10]:
pip install ffmpeg

Note: you may need to restart the kernel to use updated packages.


In [29]:
from tkinter import *
from tkinter.messagebox import showinfo
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
from tkinter.filedialog import askopenfile 

Using tkinter to create a user interface for speech recognition

In [30]:
mainwindow= Tk()
mainwindow.title('Speech Recognition')
mainwindow.geometry('500x500')
mainwindow.resizable(0, 0)
mainwindow.configure(bg='steel blue')

I used speech_rcognition library to convert speech to text

In [31]:
#Function to record voice from microphone and convert it to text
def recordvoice():
    while True:
        r = sr.Recognizer() #initialzed the speech recognizer
        with sr.Microphone() as source: #using microphone as source to recognise speech 
            r.adjust_for_ambient_noise(source, duration=0.3)  #listen for 0.3 seconds to adjust to the background/ambient noise 
            audio=r.listen(source) #listen to user input
            try:    
                text1 = r.recognize_google(audio,language="en-IN") #using google to recognize the audio and set language as english
            except:
                pass
            return text1 #returns the speech converted to text

In [32]:
#creating a function for user interface in tkinter
def SpeechToText():
    speechtotextwindow = Toplevel(mainwindow)
    speechtotextwindow.title('Speech-to-Text Converter')
    speechtotextwindow.geometry("500x500")
    speechtotextwindow.configure(bg='aquamarine')
 
    Label(speechtotextwindow, text='Speech-to-Text Converter', font=("Times New Roman", 20)).place(x=70,y=3)
 
    text = Text(speechtotextwindow, font=12, height=10, width=50)
    text.place(x=22, y=120)
   
    recordbutton = Button(speechtotextwindow, text='RECORD', bg='grey',font=('Times New Roman', 16), command=lambda: text.insert(END, recordvoice()))
    recordbutton.place(x=150, y=50)

In [33]:
#Function to convert audio files to text
#we seperate long audio files into chunks and translate each chunk
def get_large_audio(path):
    r = sr.Recognizer()
    #pydub lets us save file in any format. Here we are using .wav file format
    sound = AudioSegment.from_wav(path)  #open the audio file using AudioSegment from pydub
    # split audio sound where silence is there and get chunks
    chunks = split_on_silence(sound,
        min_silence_len = 500,
        silence_thresh = sound.dBFS-14,
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = "" 
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav") #exporting the chunks and saving it in the directory
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source) #record the file
            try:
                text = r.recognize_google(audio_listened) #recognise the audio
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                whole_text += text  #append the text in chunks to create a full text 
    return whole_text

In [34]:
#Creating a function to open the audio file in tkinter
def open_masker():
    global fileName
    fileName = filedialog.askopenfilename(filetypes=(("Audio Files", ".wav .ogg"),("All Files","*.*")))
    if fileName =="": 
        fileName = None 
    else:
        fh = open(fileName, "r")
        fh.close()   

In [35]:
#Function to read the user input audio file using the get_large_audio function 
def ReadAudio():
    return get_large_audio(fileName)

In [36]:
def AudioToText():
    audiototextwindow = Toplevel(mainwindow)
    audiototextwindow.title('Audio-to-Text Converter')
    audiototextwindow.geometry("500x500")
    audiototextwindow.configure(bg='slate blue')
 
    Label(audiototextwindow, text='Audio-to-Text Converter', font=("Times New Roman", 20)).place(x=70,y=3)
 
    text = Text(audiototextwindow, font=12, height=20, width=48)
    text.place(x=22, y=110)
    uploadbutton= Button(audiototextwindow, text='upload',font=('Times New Roman', 16), bg='grey', command=open_masker)
    uploadbutton.place(x=120, y=52)
    
    convertbutton = Button(audiototextwindow, text='convert', font=('Times New Roman', 16),bg='grey', command=lambda: text.insert(END, ReadAudio()))
    convertbutton.place(x=200, y=52)

In [37]:
Label(mainwindow, text='SPEECH RECOGNITION',
     font=('Times New Roman', 20), wrap=True, wraplength=450).place(x=70, y=5)

audiototextbutton = Button(mainwindow, text='Audio-To-Text Conversion', font=('Times New Roman', 16), bg='grey', command=AudioToText)
audiototextbutton.place(x=100, y=150)
 
speechtotextbutton = Button(mainwindow, text='Speech-To-Text Conversion', font=('Times New Roman', 16), bg='grey', command=SpeechToText)
speechtotextbutton.place(x=100, y=250)
 
mainwindow.update()
mainwindow.mainloop()